# Bronze Layer - Ingestao de Dados

Este notebook realiza a extracao de dados brutos e adiciona metadados de ingestao.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, lit
import urllib.request
import os

spark = SparkSession.builder \
    .appName("Bronze Layer") \
    .getOrCreate()


## Baixando dados do GitHub e criando diretorios


In [ ]:
url_github = "https://raw.githubusercontent.com/kauanLDD/pipeline-dados-vendas/main/data.csv"
arquivo_local = "data/source/data.csv"

os.makedirs("data/source", exist_ok=True)
os.makedirs("data/bronze", exist_ok=True)

urllib.request.urlretrieve(url_github, arquivo_local)

df_raw = spark.read.csv(arquivo_local, header=True, inferSchema=True)

print(f"Dados lidos: {df_raw.count()} linhas")


## Adicionando metadados e salvando em Parquet


In [ ]:
df_with_metadata = df_raw \
    .withColumn("data_ingestao", current_timestamp()) \
    .withColumn("fonte_arquivo", lit("github/pipeline-dados-vendas/data.csv"))

output_path = "data/bronze/dados_brutos.parquet"

df_with_metadata.write \
    .mode("overwrite") \
    .parquet(output_path)

print(f"Dados salvos em: {output_path}")

df_verificacao = spark.read.parquet(output_path)
print(f"Verificacao: {df_verificacao.count()} linhas salvas")

spark.stop()
